In [3]:
from dataclasses import dataclass
from typing import List

import numpy as np

from astropy.coordinates import SkyCoord

In [16]:
objects = ["Groth", "M102", "M71", "M27", "M29", "M52", "M110", "M74", "M76", "M45"]

kwargs = dict(
    filters = ["B", "V", "rp"],
    exposures = [120, 120, 120],
    counts = [2, 1, 1],
    binnings = [1, 1, 1],
)

# kwargs = dict(
#     filters = ["ip"],
#     exposures = [120],
#     counts = [1],
#     binnings = [1],
# )

In [17]:
@dataclass
class Plan:
    name: str
    repeat: int = 1
    filters: List[str] = None
    exposures: List[float] = None
    binnings: List[int] = None
    counts: List[int] = None
    autofocus: bool = False
    delay: int = 30
    
    @property
    def coord(self):
        if not hasattr(self, "_coord"):
            self._coord = SkyCoord.from_name(self.name)
            
        return self._coord
    
    def plan(self):
        retval = ''
        if self.autofocus:
            retval += "#autofocus\n"
            
        if self.repeat > 1:
            retval += f'#repeat {self.repeat}\n'

        retval += (
            f"#binning {','.join([str(b) for b in self.binnings])}\n"
            f"#filter {','.join(self.filters)}\n"
            f"#interval {','.join([str(e) for e in self.exposures])}\n"
            f"#count {','.join([str(c) for c in self.counts])}\n"
            f"{self.name}\t{self.coord.ra.to_string('hour', sep=':')}\t{self.coord.dec.to_string('degree', sep=':')}"
        )
        return retval
    
    @property
    def min_time(self):
        """
        Calculate the total exposure time for this plan.
        """
        per_rep = np.array(self.counts) * np.asarray(self.exposures) + self.delay
        return sum(per_rep * self.repeat)
        

In [18]:
@dataclass
class Plans:
    plans: List[Plan]
    
    @property
    def min_time(self):
        return sum([p.min_time for p in self.plans])

In [27]:
plans = {}
planos = []
for obj in objects:
    p = Plan(name=obj, **kwargs, repeat=4, autofocus=True)
    if obj == "Groth":
        p._coord = SkyCoord(ra="14:19:17.8", dec="52:49:26.5", unit=("hour", "degree"))
    plans[obj] = p.plan()
    planos.append(p)
    

In [28]:
mps = Plans(planos)

In [29]:
foo = []
for k, v in plans.items():
    foo.append(f";\n; Object {k}\n;")
    foo.append(v)
    foo.append('')

print(f"Total time is {mps.min_time / 60:.3f} sec, {mps.min_time / 60 /60:.3f} hour") 
print("\n".join(foo))

Total time is 380.000 sec, 6.333 hour
;
; Object Groth
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
Groth	14:19:17.8	52:49:26.5

;
; Object M102
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M102	15:06:29.560992	55:45:47.90988

;
; Object M71
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M71	19:53:46.48992	18:46:45.0984

;
; Object M27
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M27	19:59:36.36318501	22:43:16.31205955

;
; Object M29
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M29	20:23:46.32	38:29:13.2

;
; Object M52
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M52	23:24:46.8	61:35:24

;
; Object M110
;
#autofocus
#repeat 4
#binning 1,1,1
#filter B,V,rp
#interval 120,120,120
#count 2,1,1
M110	0:40:22.057235	41:41:07.50722014

;
; 

In [31]:
with open('ast266_color.txt', 'w') as f:
    f.write("\n".join(foo))

In [10]:
p.min_time / 60

522.5

In [11]:
mps.min_time / 60 / 60

78.375